In [1]:
import numpy as np
import json
from jinja2 import Template

import lal
import utils

In [2]:
event_list_pyring = ["S150914", "S170104", "S170814", "S170823", "S190408an", "S190512at", "S190513bm", "S190519bj", "S190521g", "S190521r", "S190602aq", "S190706ai", "S190708ap", "S190727h", "S190828j", "S190910s", "S190915ak"]
event_list_pseob = ["S190421ar", "S190503bf", "S190519bj", "S190521g", "S190521r", "S190602aq", "S190706ai", "S190727h", "S190910s"]
event_list = sorted(set(event_list_pyring + event_list_pseob))

data_file_path_template_pyring = "{event}/rin_{event}_pyring_{param}.dat.gz"
data_file_path_template_pseob = "{event}/rin_{event}_pseobnrv4hm_{param}.dat.gz"
data_file_path_template_IMR = "{event}/rin_{event}_IMR_{param}.dat.gz"

# Parameters to be tabulated
params_pyring = ["f_t_0", "tau_t_0"]
params_pseob = ["freq_220_modGR", "tau_220_modGR"]
params_IMR = ["freq_220", "tau_220"]
params = params_pyring + params_pseob + params_IMR

sample_dict = {}
# Load the posterior sample files produced by pull_data_pyring.py
for param in params_pyring:
    sample_dict[param] = {}
    for event in event_list_pyring:
        sample_dict[param][event] = np.loadtxt(data_file_path_template_pyring.format(event=event, param=param))

# Load the posterior sample files produced by pull_data_pseob.py
for param in params_pseob:
    sample_dict[param] = {}
    for event in event_list_pseob:
        sample_dict[param][event] = np.loadtxt(data_file_path_template_pseob.format(event=event, param=param))
        
# Load the posterior sample files produced by pull_data_IMR.py
for param in params_IMR:
    sample_dict[param] = {}
    for event in event_list:
        # FIXME: Also get the posterior samples for O1O2 events
        try:
            sample_dict[param][event] = np.loadtxt(data_file_path_template_IMR.format(event=event, param=param))
        except:
            pass

In [3]:
# Prepare the dictionary that should be fed to jinja2
jinja2_data_dict = []
for idx, event in enumerate(event_list):
    e = utils.Event(event)
    jinja2_data_dict.append({"catalog_id": e.name})

In [4]:
# Compute summary statistics: median, 90% CI limits
for idx, event in enumerate(event_list):
    for param in params:
        try:
            jinja2_data_dict[idx][param+"_median"] = np.median(sample_dict[param][event])
            # 5th-percentile relative to the median
            jinja2_data_dict[idx][param+"_lower_limit"] = np.median(sample_dict[param][event]) - np.percentile(sample_dict[param][event], 5)
            # 95th-percentile relative to the median
            jinja2_data_dict[idx][param+"_upper_limit"] = np.percentile(sample_dict[param][event], 95) - np.median(sample_dict[param][event])
        except:
            # That parameter does not exist in the dict, returning np.nan
            jinja2_data_dict[idx][param+"_median"] = None
            # 5th-percentile relative to the median
            jinja2_data_dict[idx][param+"_lower_limit"] = None
            # 95th-percentile relative to the median
            jinja2_data_dict[idx][param+"_upper_limit"] = None

In [5]:
# Format the output, and show a dash for nan
for idx, event in enumerate(event_list):
    # IMR
    try:
        jinja2_data_dict[idx]["freq_IMR"] = "${0:.0f}^{{+{1:.0f}}}_{{-{2:.0f}}}$".format(jinja2_data_dict[idx]["freq_220_median"], jinja2_data_dict[idx]["freq_220_upper_limit"], jinja2_data_dict[idx]["freq_220_lower_limit"])
        jinja2_data_dict[idx]["tau_IMR"] = "${0:.1f}^{{+{1:.1f}}}_{{-{2:.1f}}}$".format(1e3*jinja2_data_dict[idx]["tau_220_median"], 1e3*jinja2_data_dict[idx]["tau_220_upper_limit"], 1e3*jinja2_data_dict[idx]["tau_220_lower_limit"])
    except:
        jinja2_data_dict[idx]["freq_IMR"] = "$-$"
        jinja2_data_dict[idx]["tau_IMR"] = "$-$"
    # pyRing
    try:
        jinja2_data_dict[idx]["freq_pyRing"] = "${0:.0f}^{{+{1:.0f}}}_{{-{2:.0f}}}$".format(jinja2_data_dict[idx]["f_t_0_median"], jinja2_data_dict[idx]["f_t_0_upper_limit"], jinja2_data_dict[idx]["f_t_0_lower_limit"])
        jinja2_data_dict[idx]["tau_pyRing"] = "${0:.1f}^{{+{1:.1f}}}_{{-{2:.1f}}}$".format(1e3*jinja2_data_dict[idx]["tau_t_0_median"], 1e3*jinja2_data_dict[idx]["tau_t_0_upper_limit"], 1e3*jinja2_data_dict[idx]["tau_t_0_lower_limit"])
    except:
        jinja2_data_dict[idx]["freq_pyRing"] = "$-$"
        jinja2_data_dict[idx]["tau_pyRing"] = "$-$"
    # pSEOB
    try:
        jinja2_data_dict[idx]["freq_pSEOB"] = "${0:.0f}^{{+{1:.0f}}}_{{-{2:.0f}}}$".format(jinja2_data_dict[idx]["freq_220_modGR_median"], jinja2_data_dict[idx]["freq_220_modGR_upper_limit"], jinja2_data_dict[idx]["freq_220_modGR_lower_limit"])
        jinja2_data_dict[idx]["tau_pSEOB"] = "${0:.1f}^{{+{1:.1f}}}_{{-{2:.1f}}}$".format(1e3*jinja2_data_dict[idx]["tau_220_modGR_median"], 1e3*jinja2_data_dict[idx]["tau_220_modGR_upper_limit"], 1e3*jinja2_data_dict[idx]["tau_220_modGR_lower_limit"])
    except:
        jinja2_data_dict[idx]["freq_pSEOB"] = "$-$"
        jinja2_data_dict[idx]["tau_pSEOB"] = "$-$"

In [6]:
# Jinja2 template for the table in LaTeX
LaTex_table_jinja_template = r"""
\begin{tabular}{llllllll}
\toprule
Event & \multicolumn{3}{c}{Redshifted} & \hphantom{X} & \multicolumn{3}{c}{Redshifted} \\
& \multicolumn{3}{c}{frequency [Hz]} & \hphantom{X} & \multicolumn{3}{c}{damping time [ms]} \\[0.075cm]
\cline{2-4}
\cline{6-8}
& IMR & DS & pSEOB & \hphantom{X} & IMR & DS & pSEOB \\
\midrule
{% for event in  jinja2_data_dict %}
{{ event.catalog_id }} &
{{ event.freq_IMR }} &
{{ event.freq_pyRing }} &
{{ event.freq_pSEOB }} &
\hphantom{X} &
{{ event.tau_IMR }} &
{{ event.tau_pyRing }} &
{{ event.tau_pSEOB }}
\\[0.075cm]
{% endfor %}
\bottomrule
\end{tabular}
"""

In [7]:
Template(LaTex_table_jinja_template).stream(jinja2_data_dict=jinja2_data_dict).dump("tab_rin_freq_tau_results.tex")